In [ ]:
!pip install -q langchain-google-genai cassio datasets langchain tiktoken PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 443.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
!pip install langchain -U langchain-community
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

from datasets import load_dataset

import cassio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

In [ ]:
#to connect to astra db
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:YKYxiZuMbgEtOxHAFuHrPxxr:e4e411f5cd0654edba5beb05b1ac581f538c7fe29bbd1e4687a13efcc5b392b1"
ASTRA_DB_ID = "a9f31c20-1621-41a0-b766-9f71059e4ffc"

In [ ]:
PdfReader = PdfReader('/content/budget_speech.pdf')

In [ ]:
from typing_extensions import Concatenate
#read text from pdf

raw_text = ''
for i, page in enumerate(PdfReader.pages):
  text = page.extract_text()
  if text:
    raw_text += text

In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
GOOGLE_API_KEY = "AIzaSyD790EJnRTWPHILMzk4ghfSUJu0VBbcMsU0"

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)
embedding = GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY, model="models/embedding-001")

In [ ]:
astra_vector_store = Cassandra(
    embedding=embedding,
    session=None,
    keyspace=None,
    table_name="qa_mini_demo",
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
# text split without increasaing token size
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines. " % len(texts[:50]) )
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines. 


In [ ]:
first_q = True
while True:
  if first_q:
    query_txt = input("\nEnter Your Question: (Type 'q' to exit): ".strip())

  else:
    query_txt = input("\nYour Next Question: (Type 'q' to exit): ".strip())

  if query_txt.lower()=="q":
    break

  if query_txt == "":
    continue

  first_q = False

  print("\nQUESTION: \"%s\"" % query_txt)
  answer = astra_vector_index.query(query_txt, llm=llm).strip()
  print("\nANSWER: \"%s\"" % astra_vector_index.query(query_txt, llm=llm))

  print("RELEVANT DOCUMENTS: ")
  for doc, score in astra_vector_store.similarity_search_with_score(query_txt, k=4):
    print(" [%0.4f] \"%s ...\"" % (score, doc.page_content[:80]))

Enter Your Question: (Type 'q' to exit):gdp of india?

QUESTION: "gdp of india?"



ANSWER: "The provided text mentions that India's capital investment outlay is 3.3 percent of GDP, and that the GDP growth is estimated to be 7 percent.  However, the actual GDP amount is not specified."
RELEVANT DOCUMENTS: 


 [0.8445] "estimated to be at 7 per cent. It is notable that this is the highest among all  ..."
 [0.8332] "multiplier impact on growth and employment. After the subdued period of 
the pan ..."
 [0.8311] "With the theme of ‘ Vasudhaiva Kutumbakam’ , we are steering an 
ambitious, peop ..."
 [0.8298] "components of the scheme will include not only financial support 
but also acces ..."
Your Next Question: (Type 'q' to exit):anything related to women's development?

QUESTION: "anything related to women's development?"



ANSWER: "Yes, the provided text mentions that a component of a scheme will benefit women, along with Scheduled Castes, Scheduled Tribes, OBCs, and people belonging to weaker sections.  The text does not, however, provide details about the specific scheme or the nature of the benefits."
RELEVANT DOCUMENTS: 


 [0.8375] "pedagogy, curriculum transaction, continuous professional development, 
dipstick ..."
 [0.8370] "covering 500 blocks for saturation of essential government services across 
mult ..."
 [0.8366] "components of the scheme will include not only financial support 
but also acces ..."
 [0.8351] "Pharma Innovation  
30. A new programme to promote research and innovation in 
p ..."
Your Next Question: (Type 'q' to exit):q
